
<img src="https://learn.wqu.edu/assets/39cecb6f6bdccec67223.svg" alt="Unimarconi logo" width="180"/>


---
### **COURSE OF STUDY: Masters in Financial Engineering**

**GROUP CAPSTONE PROJECT**

**Students Name:**  OKON PRINCE, FRACKSON MAKWANGWALA, PAIDAMOYO MUTEPFA

**Group ID:**  22170854

**PROJECT:** Machine Learning-Based Swing Trading Strategy for Cryptocurrencies

**Emails:** okonp07@gmail.com, paidamoyomutepfa@gmail.com, fracksonmakwangwala@gmail.com

---

In [1]:
!pip install ta

In [2]:
!pip install backtrader


In [3]:
# Data Manipulation and Analysis
import pandas as pd
import numpy as np
from pandas_datareader import data as pdr


# Machine Learning Algorithms
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split, TimeSeriesSplit, GridSearchCV, RandomizedSearchCV

# Time-Series Analysis
import statsmodels.api as sm
import statsmodels.tsa.api as tsa

# Technical Indicators
import ta

# Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Backtesting
import backtrader as bt

# Sentiment Analysis (Twitter API)
import tweepy

# News API
import requests

# Macroeconomic Data (FRED)
from pandas_datareader import data as pdr

#Crypto Data
import yfinance as yf

# Additional Libraries
import json
import datetime
import re


In [4]:
#  Define the start and end date for data collection
start_date = '2014-01-01'
end_date = datetime.datetime.now().strftime('%Y-%m-%d')

# Function to fetch cryptocurrency data
def fetch_crypto_data(ticker, start, end):
    """
    Fetch historical cryptocurrency price data from Yahoo Finance using yfinance.
    """
    try:
        data = yf.download(ticker, start=start, end=end)
        return data
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return None

# Fetch Bitcoin (BTC) data
btc_data = fetch_crypto_data('BTC-USD', start_date, end_date)

# Display the first few rows of BTC data
if btc_data is not None:
   print(btc_data.head())
else:
    print("Failed to retrieve data.")


[*********************100%***********************]  1 of 1 completed

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2014-09-17  465.864014  468.174011  452.421997  457.334015  457.334015   
2014-09-18  456.859985  456.859985  413.104004  424.440002  424.440002   
2014-09-19  424.102997  427.834991  384.532013  394.795990  394.795990   
2014-09-20  394.673004  423.295990  389.882996  408.903992  408.903992   
2014-09-21  408.084991  412.425995  393.181000  398.821014  398.821014   

              Volume  
Date                  
2014-09-17  21056800  
2014-09-18  34483200  
2014-09-19  37919700  
2014-09-20  36863600  
2014-09-21  26580100  


In [5]:
# Check for missing values
print(btc_data.isnull().sum())

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64


In [6]:
# Add RSI
btc_data['RSI'] = ta.momentum.RSIIndicator(btc_data['Close']).rsi()

# Add Bollinger Bands
bb = ta.volatility.BollingerBands(btc_data['Close'])
btc_data['BB_bbm'] = bb.bollinger_mavg()
btc_data['BB_bbh'] = bb.bollinger_hband()
btc_data['BB_bbl'] = bb.bollinger_lband()

# Add MACD
macd = ta.trend.MACD(btc_data['Close'])
btc_data['MACD'] = macd.macd()
btc_data['MACD_signal'] = macd.macd_signal()


In [7]:
# Define the start and end date for data collection
start_date = '2015-01-01'
end_date = datetime.datetime.now().strftime('%Y-%m-%d')

# Fetch Gold Price Data
def fetch_data(ticker, start, end):
    try:
        data = yf.download(ticker, start=start, end=end)
        return data
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return None

# Fetch data for Gold, Dollar Index, S&P 500, and VIX
gold_data = fetch_data('GC=F', start_date, end_date)
dollar_index_data = fetch_data('DX-Y.NYB', start_date, end_date)
sp500_data = fetch_data('^GSPC', start_date, end_date)
vix_data = fetch_data('^VIX', start_date, end_date)

# Define FRED ticker for US Interest Rates (e.g., Federal Funds Effective Rate)
def fetch_interest_rates(ticker, start, end):
    try:
        data = pdr.get_data_fred(ticker, start, end)
        return data
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return None

# Fetch interest rate data
interest_rates_data = fetch_interest_rates('FEDFUNDS', start_date, end_date)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [8]:
# Example of using Twitter API
def fetch_twitter_sentiment(query, count=100):
    # Implementation to fetch tweets and analyze sentiment
    pass


##Align and Merge Data
Resample and merge the BTC data with the gold and dollar index data. Ensure that all datasets are aligned on the same dates.

In [9]:
# Ensure the index is a DateTime index
btc_data.index = pd.to_datetime(btc_data.index)

# Drop columns that are not neccesary for our work
btc_data = btc_data.drop(columns=['Open', 'High', 'Low', 'Adj Close'])

# Remove data points before the specified date
btc_data = btc_data.loc['2015-01-01':]

# Display the first few rows of the cleaned data
btc_data.head()



,Close,Volume,RSI,BB_bbm,BB_bbh,BB_bbl,MACD,MACD_signal
Date,,,,,,,,
2015-01-01,314.248993,8036550,40.192392,324.905199,348.403326,301.407071,-10.733248,-11.073772
2015-01-02,315.032013,7860650,40.706619,323.287999,344.740485,301.835513,-10.378369,-10.934692
2015-01-03,281.082001,33054400,29.045723,319.760500,344.378157,295.142842,-12.690322,-11.285818
2015-01-04,264.195007,55629100,25.181629,315.703000,347.746210,283.659790,-15.704172,-12.169488
2015-01-05,274.473999,43962800,31.182920,313.073599,349.312708,276.834491,-17.066509,-13.148892


In [10]:
# Keep only the 'Close' prices and reset index to have a Date column
# Keep all columns and rename the 'Close' column to 'BTC_Close'
btc_data = btc_data.rename(columns={'Close': 'BTC_Close'})
gold_data = gold_data[['Close']].rename(columns={'Close': 'Gold_Close'})
dollar_index_data = dollar_index_data[['Close']].rename(columns={'Close': 'Dollar_Index_Close'})
sp500_data = sp500_data[['Close']].rename(columns={'Close': 'SP500_Close'})
vix_data = vix_data[['Close']].rename(columns={'Close': 'VIX_Close'})

In [11]:
# Merge all dataframes on the 'Date' column
merged_data = pd.merge(btc_data, gold_data, on='Date', how='outer')
merged_data = pd.merge(merged_data, dollar_index_data, on='Date', how='outer')
merged_data = pd.merge(merged_data, sp500_data, on='Date', how='outer')
merged_data = pd.merge(merged_data, vix_data, on='Date', how='outer')

# Display the first few rows of the merged data
print(merged_data.head())


             BTC_Close    Volume        RSI      BB_bbm      BB_bbh  \
Date                                                                  
2015-01-01  314.248993   8036550  40.192392  324.905199  348.403326   
2015-01-02  315.032013   7860650  40.706619  323.287999  344.740485   
2015-01-03  281.082001  33054400  29.045723  319.760500  344.378157   
2015-01-04  264.195007  55629100  25.181629  315.703000  347.746210   
2015-01-05  274.473999  43962800  31.182920  313.073599  349.312708   

                BB_bbl       MACD  MACD_signal   Gold_Close  \
Date                                                          
2015-01-01  301.407071 -10.733248   -11.073772          NaN   
2015-01-02  301.835513 -10.378369   -10.934692  1186.000000   
2015-01-03  295.142842 -12.690322   -11.285818          NaN   
2015-01-04  283.659790 -15.704172   -12.169488          NaN   
2015-01-05  276.834491 -17.066509   -13.148892  1203.900024   

            Dollar_Index_Close  SP500_Close  VIX_Close  
Dat

In [12]:
df=merged_data
df.head(10)

,BTC_Close,Volume,RSI,BB_bbm,BB_bbh,BB_bbl,MACD,MACD_signal,Gold_Close,Dollar_Index_Close,SP500_Close,VIX_Close
Date,,,,,,,,,,,,
2015-01-01,314.248993,8036550,40.192392,324.905199,348.403326,301.407071,-10.733248,-11.073772,NaN,NaN,NaN,NaN
2015-01-02,315.032013,7860650,40.706619,323.287999,344.740485,301.835513,-10.378369,-10.934692,1186.000000,91.080002,2058.199951,17.790001
2015-01-03,281.082001,33054400,29.045723,319.760500,344.378157,295.142842,-12.690322,-11.285818,NaN,NaN,NaN,NaN
2015-01-04,264.195007,55629100,25.181629,315.703000,347.746210,283.659790,-15.704172,-12.169488,NaN,NaN,NaN,NaN
2015-01-05,274.473999,43962800,31.182920,313.073599,349.312708,276.834491,-17.066509,-13.148892,1203.900024,91.379997,2020.579956,19.920000
2015-01-06,286.188995,23245700,37.350701,311.394249,349.309471,273.479027,-17.004847,-13.920083,1219.300049,91.500000,2002.609985,21.120001
2015-01-07,294.337006,24866800,41.291870,310.541299,349.178637,271.903962,-16.112766,-14.358620,1210.599976,91.889999,2025.900024,19.309999
2015-01-08,283.348999,19982500,37.835207,308.816600,349.043045,268.590156,-16.106756,-14.708247,1208.400024,92.370003,2062.139893,17.010000
2015-01-09,290.407990,18718600,41.238458,306.839200,346.600072,267.078327,-15.355385,-14.837675,1216.000000,91.940002,2044.810059,17.549999


because some of our indices only have values for trading days, we have a lot of Null values in the weekends. We will use the Foward fill method to fill out these values

In [13]:
# Forward-fill all NaN values in the DataFrame
df.ffill(inplace=True)
df.tail(10)

,BTC_Close,Volume,RSI,BB_bbm,BB_bbh,BB_bbl,MACD,MACD_signal,Gold_Close,Dollar_Index_Close,SP500_Close,VIX_Close
Date,,,,,,,,,,,,
2024-09-07,54139.687500,19061486526,35.510142,59382.483789,65113.049866,53651.917712,-1475.666689,-870.924084,2493.500000,101.180000,5408.419922,22.379999
2024-09-08,54841.566406,18268287531,38.144589,59149.889453,65211.614485,53088.164421,-1568.015146,-1010.342296,2493.500000,101.180000,5408.419922,22.379999
2024-09-09,57019.535156,34618096173,45.574364,59050.226562,65182.820010,52917.633115,-1448.757710,-1098.025379,2501.800049,101.550003,5471.049805,19.450001
2024-09-10,57648.710938,28857630507,47.534894,58873.902539,64954.535388,52793.269691,-1288.621678,-1136.144639,2512.300049,101.629997,5495.520020,19.080000
2024-09-11,57343.171875,37049062672,46.655960,58721.965430,64796.137293,52647.793567,-1172.847313,-1143.485174,2512.100098,101.680000,5554.129883,17.690001
2024-09-12,58127.011719,33835707949,49.248572,58423.598242,63976.773095,52870.423390,-1006.246586,-1116.037456,2551.199951,101.370003,5595.759766,17.070000
2024-09-13,60571.300781,32490528356,56.369714,58243.213672,63243.734636,53242.692707,-669.265731,-1026.683111,2581.300049,101.110001,5626.020020,16.559999
2024-09-14,60005.121094,16428405496,54.463408,58026.792578,62271.824842,53781.760314,-442.787764,-909.904042,2581.300049,101.110001,5626.020020,16.559999
2024-09-15,59182.835938,18120960867,51.727383,57841.901367,61507.803853,54175.998881,-325.897143,-793.102662,2581.300049,101.110001,5626.020020,16.559999


In [14]:
#Re-order the data frame to place BTC Close at the end of our data frame
df = df[[col for col in df.columns if col != 'BTC_Close'] + ['BTC_Close']]
df.head()

,Volume,RSI,BB_bbm,BB_bbh,BB_bbl,MACD,MACD_signal,Gold_Close,Dollar_Index_Close,SP500_Close,VIX_Close,BTC_Close
Date,,,,,,,,,,,,
2015-01-01,8036550,40.192392,324.905199,348.403326,301.407071,-10.733248,-11.073772,NaN,NaN,NaN,NaN,314.248993
2015-01-02,7860650,40.706619,323.287999,344.740485,301.835513,-10.378369,-10.934692,1186.000000,91.080002,2058.199951,17.790001,315.032013
2015-01-03,33054400,29.045723,319.760500,344.378157,295.142842,-12.690322,-11.285818,1186.000000,91.080002,2058.199951,17.790001,281.082001
2015-01-04,55629100,25.181629,315.703000,347.746210,283.659790,-15.704172,-12.169488,1186.000000,91.080002,2058.199951,17.790001,264.195007
2015-01-05,43962800,31.182920,313.073599,349.312708,276.834491,-17.066509,-13.148892,1203.900024,91.379997,2020.579956,19.920000,274.473999


Our goal is to predict the direction of movement of BTC. to doi this, we will first change our target from a continous variable (the continour prices of BTC) to a categorical variable (A boolean representation of the direcrtion of the movement of BTC), with 1 representing a positive price change and 0 representing a Negative price change

In [15]:
# Create a new column 'BTC_Direction' to indicate the direction of movement
df['BTC_Direction'] = (df['BTC_Close'].diff() > 0).astype(int)


<ipython-input-15-ede1e335d17f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['BTC_Direction'] = (df['BTC_Close'].diff() > 0).astype(int)


In [16]:
df.head()

,Volume,RSI,BB_bbm,BB_bbh,BB_bbl,MACD,MACD_signal,Gold_Close,Dollar_Index_Close,SP500_Close,VIX_Close,BTC_Close,BTC_Direction
Date,,,,,,,,,,,,,
2015-01-01,8036550,40.192392,324.905199,348.403326,301.407071,-10.733248,-11.073772,NaN,NaN,NaN,NaN,314.248993,0
2015-01-02,7860650,40.706619,323.287999,344.740485,301.835513,-10.378369,-10.934692,1186.000000,91.080002,2058.199951,17.790001,315.032013,1
2015-01-03,33054400,29.045723,319.760500,344.378157,295.142842,-12.690322,-11.285818,1186.000000,91.080002,2058.199951,17.790001,281.082001,0
2015-01-04,55629100,25.181629,315.703000,347.746210,283.659790,-15.704172,-12.169488,1186.000000,91.080002,2058.199951,17.790001,264.195007,0
2015-01-05,43962800,31.182920,313.073599,349.312708,276.834491,-17.066509,-13.148892,1203.900024,91.379997,2020.579956,19.920000,274.473999,1


In [17]:
# Drop the first row with NaN values explicitly
df = df.drop(index=df.index[0])

# Drop the BTC_Close column to avoid data leakage
df = df.drop(columns=['BTC_Close'])

df.head()

,Volume,RSI,BB_bbm,BB_bbh,BB_bbl,MACD,MACD_signal,Gold_Close,Dollar_Index_Close,SP500_Close,VIX_Close,BTC_Direction
Date,,,,,,,,,,,,
2015-01-02,7860650,40.706619,323.287999,344.740485,301.835513,-10.378369,-10.934692,1186.000000,91.080002,2058.199951,17.790001,1
2015-01-03,33054400,29.045723,319.760500,344.378157,295.142842,-12.690322,-11.285818,1186.000000,91.080002,2058.199951,17.790001,0
2015-01-04,55629100,25.181629,315.703000,347.746210,283.659790,-15.704172,-12.169488,1186.000000,91.080002,2058.199951,17.790001,0
2015-01-05,43962800,31.182920,313.073599,349.312708,276.834491,-17.066509,-13.148892,1203.900024,91.379997,2020.579956,19.920000,1
2015-01-06,23245700,37.350701,311.394249,349.309471,273.479027,-17.004847,-13.920083,1219.300049,91.500000,2002.609985,21.120001,1
